# Imports

In [1]:
from collections import deque
from typing import List, Tuple, Optional
from src.base import MazeSolverBase, MazeEnvironment
from src.generator import StandardMazeGenerator,FaultyMazeGenerator
from src.validator import MazeJudge

# Function Skeleton

In [2]:
class MyBFSPathfinder(MazeSolverBase):
    """
    Example of a learner's implementation using BFS.
    They only use methods from the 'env' object.
    """
    def search(self, env: MazeEnvironment) -> Optional[List[Tuple[int, int]]]:
        start = env.get_start()
        goal = env.get_end()
        
        # Standard BFS setup
        queue = deque([(start, [start])])
        visited = {start}
        
        while queue:
            current, path = queue.popleft()
            
            if current == goal:
                return path
            
            # The learner discovers neighbors without seeing the whole grid matrix
            for neighbor in env.get_successors(current):
                if neighbor not in visited:
                    visited.add(neighbor)
                    queue.append((neighbor, path + [neighbor]))
        
        return None

In [3]:
class StraightPathfinder(MazeSolverBase):
    """
    A suboptimal pathfinder that prioritizes directions: Up, Left, Down, Right.
    It stops if it hits a dead end or exceeds the maximum possible steps.
    """
    def search(self, env: MazeEnvironment) -> Optional[List[Tuple[int, int]]]:
        start = env.get_start()
        goal = env.get_end()
        
        path = [start]
        current = start
        visited = {start}

        # Set max iterations to the total area of the maze
        # This ensures we don't loop forever but have enough steps to visit every cell
        max_iterations = env.rows * env.cols

        for _ in range(max_iterations):
            if current == goal:
                return path

            successors = env.get_successors(current)
            
            # Priority order: Up, Left, Down, Right
            priorities = [
                (current[0] + 1, current[1]),
                (current[0], current[1] + 1),
                (current[0] - 1, current[1]),
                (current[0], current[1] - 1)
            ]

            next_move = None
            for move in priorities:
                # Check if the move is valid and not already visited
                if move in successors and move not in visited:
                    next_move = move
                    break
            
            if next_move:
                current = next_move
                path.append(current)
                visited.add(current)
            else:
                # No unvisited neighbors found (dead end)
                return None

        return None

In [4]:
class EmptyPathfinder(MazeSolverBase):
    """
    Empty Pathfinder for Error of not implement the method
    """
    pass

In [5]:
class HillClimbingPathfinder(MazeSolverBase):
    """
    Learner's implementation using Hill Climbing (Greedy Search).
    This algorithm always moves to the neighbor with the lowest heuristic value.
    It does NOT backtrack; if it reaches a local optimum or dead end, it fails.
    """
    def search(self, env: MazeEnvironment) -> Optional[List[Tuple[int, int]]]:
        start = env.get_start()
        goal = env.get_end()
        
        # Initialize path and current position
        path = [start]
        current = start
        visited = {start}
        
        max_steps = env.rows * env.cols
        
        for _ in range(max_steps):
            if current == goal:
                return path
            
            successors = env.get_successors(current)

            unvisited_neighbors = [n for n in successors if n not in visited]
            
            if not unvisited_neighbors:
                # Dead end: no unvisited neighbors to move to
                return path
            
            next_node = min(unvisited_neighbors, key=lambda n: env.get_heuristic(n))
            
            current_h = env.get_heuristic(current)
            next_h = env.get_heuristic(next_node)
            
            current = next_node
            path.append(current)
            visited.add(current)
            
        return path

# Setting up Quiz

In [22]:
generator = FaultyMazeGenerator()
judge = MazeJudge(generator)
pathfinders = [MyBFSPathfinder,StraightPathfinder,EmptyPathfinder,HillClimbingPathfinder]
solver_idx = 0
solver = pathfinders[solver_idx]()

In [23]:
generator.generate_fixed().get_grid()

[[0, 0, 0, 0, 0],
 [0, 1, 1, 0, 0],
 [0, 0, 0, 0, 0],
 [0, 0, 1, 0, 0],
 [0, 0, 1, 0, 0]]

In [24]:
print(solver.search(generator.generate_fixed()))

[(0, 0), (0, 1), (0, 2), (0, 3), (0, 4), (1, 4), (2, 4), (3, 4), (4, 4)]


# Validation

In [29]:
print("=== SCENARIO 1: DEBUGGING WITH FIXED MAZE ===")
# In fixed mode, learners can print the grid to see what they are doing
fixed_env = generator.generate_fixed()
print("Start:", fixed_env.get_start())
print("Goal:", fixed_env.get_end())

# Debugging: Learner can see the matrix here
grid_matrix = fixed_env.get_grid()
for row in grid_matrix:
    print(row)

path = solver.search(fixed_env)
judge.validate(path, fixed_env, mode="optimal")

print("\n=== SCENARIO 2: BLIND VALIDATION (QUIZ MODE) ===")
# The judge runs the solver on a random maze.
# If the learner's search() calls env.get_grid(), it will crash with a PermissionError.
judge.test_on_random(solver,num_tests=5,seed=51)

print("\n=== SCENARIO 3: TESTING HEURISTICS ===")
# Example showing that the environment provides Manhattan distance automatically
some_pos = (2, 2)
h_value = fixed_env.get_heuristic(some_pos)
print(f"Manhattan distance from {some_pos} to goal: {h_value}")

=== SCENARIO 1: DEBUGGING WITH FIXED MAZE ===
Start: (0, 0)
Goal: (4, 4)
[0, 0, 0, 0, 0]
[0, 1, 1, 0, 0]
[0, 0, 0, 0, 0]
[0, 0, 1, 0, 0]
[0, 0, 1, 0, 0]

=== SCENARIO 2: BLIND VALIDATION (QUIZ MODE) ===
--- Running 5 Random Test(s) ---
✅ Test 1/5 passed!
✅ Test 2/5 passed!
❌ Test 3/5 failed.


ValueError: Validation Failed on test 3: The result must be a valid list of (r, c) tuples.

In [32]:
for i in range(1,5+1):
    print("at iteration",i,"having heuristic at",some_pos,"as",fixed_env.get_heuristic(some_pos))

at iteration 1 having heuristic at (2, 2) as 4.499451381944013
at iteration 2 having heuristic at (2, 2) as 18.865364794348768
at iteration 3 having heuristic at (2, 2) as 18.581090470468066
at iteration 4 having heuristic at (2, 2) as 19.640722989675506
at iteration 5 having heuristic at (2, 2) as 9.500193208074812
